# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# Load training dataset

In [ ]:
train_dataframe = pd.read_csv('dataset/1. istanbul/train_data.csv')

# Visualize some training data

In [ ]:
train_dataframe.head()

# Prepare training data

In [ ]:
train_x = train_dataframe.drop(['Subject ID', 'Class information', 'UPDRS'], axis=1)
train_x = train_x.as_matrix()
print(train_x.shape)

# Prepare training labels

In [ ]:
train_y = train_dataframe['Class information']
train_y = train_y.as_matrix()
print(train_y.shape)
train_y = np_utils.to_categorical(train_y)
print(train_y.shape)

# Prepare test data

In [ ]:
test_dataframe = pd.read_csv('dataset/1. istanbul/test_data.csv')
test_x = test_dataframe.drop(['Subject ID', 'Class information'], axis=1)
test_x = test_x.as_matrix()
print(test_x.shape)

# Prepare test labels

In [ ]:
test_y = test_dataframe['Class information']
test_y = test_y.as_matrix()
print(test_y.shape)
test_y = np_utils.to_categorical(test_y)
print(test_y.shape)

# Normalize data

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)
pd.DataFrame(train_x).head()

# Prepare model for training

In [ ]:
x = Input(shape=(train_x.shape[1],))

y = Dense(units = 64, activation='relu', input_dim=train_x.shape[1])(x)
y = Dropout(0.5)(y)

y = Dense(units = 16, activation='relu', input_dim=train_x.shape[1])(y)
y = Dropout(0.5)(y)

y = Dense(units = 2, activation='softmax')(y)

model = Model(x, y)
model.summary()
SVG(model_to_dot(model, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Train the model

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='Tensorboard2/' + date + '/')
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=5000, batch_size=20, callbacks=[tensorboard_callback])

# The End